<a href="https://colab.research.google.com/github/WiemHAD/Brief_NLP/blob/main/rephrase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Downloading librairies**

In [7]:
! pip install credentials
! pip install config

In [3]:
! pip install git+https://github.com/PrithivirajDamodaran/Parrot.git

  Cloning https://github.com/PrithivirajDamodaran/Parrot.git to /tmp/pip-req-build-8go3cfkz
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot.git /tmp/pip-req-build-8go3cfkz
     |████████████████████████████████| 2.6 MB 32.1 MB/s 
     |████████████████████████████████| 1.2 MB 71.5 MB/s 
     |████████████████████████████████| 50 kB 4.2 MB/s 
     |████████████████████████████████| 85 kB 5.2 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 3.3 MB 51.4 MB/s 
     |████████████████████████████████| 895 kB 70.0 MB/s 
     |████████████████████████████████| 636 kB 68.5 MB/s 
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8695 sha256=7a94be434b2849ffd22aa983aac367cbe075c81562f96b80d5edd4ff951b9f03
  Stored in directory: /tmp/pip-ephem-wheel-cache-gq4gdt8t/wheels/d8/27/d8/b757755a5b0b804c59714fdd8a4e2733c20802f638f17f7d96
  Created wheel for python-Levenshtein: filename=python_Levenshtein

**Import librairies**

In [8]:
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")
import pandas as pd 
import tweepy as tw
import config
from credentials import *
import re  # For preprocessing
import nltk
nltk.download('punkt')
from nltk import tokenize

In [9]:
# For reproducibility
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)

In [10]:
#Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=True)

**Scraping last tweet**

In [24]:
#twitter tokens
consumer_key= 'HxklRvkv2hhS2AUkXhEQBxpvZ'
consumer_secret= 'UEgBbSDtQzAty1EJjmy4Vjo3oAKwYfhQDkrkhRlLpk0BhCYmsv'
access_token = '1409627133253718024-StgjAUt98nEYufIr3qfm0KoCVfEkR0'
access_token_secret = 'AnmmTQZWxndzXu1lsSCNyrCM1BwCVA12aF0Eet48sx7Ft'

In [25]:
userID = '@BarackObama'


auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count= 200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )

In [107]:
last_tweet = tweets[0].full_text
last_tweet

"I could not be prouder of the incredible athletes representing @TeamUSA at the Olympics. Their determination and hard work got them this far, and they’re ready to seize this moment. We'll be rooting for them all the way."

In [79]:
last_tweets = []
for info in tweets[:10]:
    print("ID: {}".format(info.id))
    print(info.created_at)
    print(info.full_text)
    print("\n")
    last_tweets.append(info.full_text)

ID: 1418715270961500163
2021-07-23 23:30:53
I could not be prouder of the incredible athletes representing @TeamUSA at the Olympics. Their determination and hard work got them this far, and they’re ready to seize this moment. We'll be rooting for them all the way.


ID: 1418244880246075392
2021-07-22 16:21:43
Since Bruce @springsteen and I first sat down to record our podcast Renegades: Born in the USA, folks have asked about the stories we told and the people and places that shaped us, and we’re excited for this new book to continue the conversation we started last summer. https://t.co/h0FuZyNJQc


ID: 1416802924424957955
2021-07-18 16:51:54
Even when he was confined to the darkness of a jail cell, Madiba’s light shone brightly. Today, as we reflect on his legacy, I see Mandela’s spirit alive in our @ObamaFoundation leaders. I’m inspired by their service, and the work they’re doing to create the change we need now. https://t.co/mvXFZCbYNM


ID: 1416480269800939521
2021-07-17 19:29:47


In [69]:
last_tweets[0]

"I could not be prouder of the incredible athletes representing @TeamUSA at the Olympics. Their determination and hard work got them this far, and they’re ready to seize this moment. We'll be rooting for them all the way."

In [70]:
#phrases = last_tweets[0]
phrases= "I could not be prouder of the incredible athletes representing at the Olympics"

In [71]:
#for phrase in phrases:
regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
regex1 = '\@[A-Za-z0-9.-]+'

sentences = re.sub(regex, '', last_tweets[0], flags=re.MULTILINE)
sentences = re.sub(regex1,'', last_tweets[0], flags=re.MULTILINE)
print(sentences)

I could not be prouder of the incredible athletes representing  at the Olympics. Their determination and hard work got them this far, and they’re ready to seize this moment. We'll be rooting for them all the way.


In [72]:
type(sentences)

str

In [73]:
phrases = tokenize.sent_tokenize(sentences)

In [74]:
phrases

['I could not be prouder of the incredible athletes representing  at the Olympics.',
 'Their determination and hard work got them this far, and they’re ready to seize this moment.',
 "We'll be rooting for them all the way."]

In [103]:
#rephrased_tweets= []
with open('recap.txt', 'w') as f:
  for txt in phrases:
  #for phrase in phrases:
    print("-"*100)
    print("Input_phrase: ", txt)
    print("-"*100)
    f.write(txt)
    f.write('\n')
    para_phrases = parrot.augment(input_phrase=txt)
    for para_phrase in para_phrases:
      print(para_phrase)
      f.write(str(para_phrase))
      f.write('\n')

----------------------------------------------------------------------------------------------------
Input_phrase:  I could not be prouder of the incredible athletes representing  at the Olympics.
----------------------------------------------------------------------------------------------------
('i could not be prouder of the amazing athletes at the olympics', 36)
("i can't be prouder of the incredible athletes who competed in the olympics", 34)
('i could not be prouder of the fantastic athletes who represent the olympics', 33)
('i could not be prouder of the incredible athletes who performed at the olympics', 26)
("i couldn't be prouder of the incredible athletes who represent us at the olympics", 23)
("i couldn't be prouder of the incredible athletes who represent me at the olympics", 23)
("i couldn't be prouder of the incredible athletes representing us at the olympics", 17)
('i could not be prouder of the incredible athletes representing us at the olympics', 15)
-----------------

**Class**

In [115]:
class rephrase:
  def __init__(self,userID):
    self.userID = userID


  def scraping(userID):
    '''scraping last 200 tweets of a designed userId'''

    consumer_key= 'HxklRvkv2hhS2AUkXhEQBxpvZ'
    consumer_secret= 'UEgBbSDtQzAty1EJjmy4Vjo3oAKwYfhQDkrkhRlLpk0BhCYmsv'
    access_token = '1409627133253718024-StgjAUt98nEYufIr3qfm0KoCVfEkR0'
    access_token_secret = 'AnmmTQZWxndzXu1lsSCNyrCM1BwCVA12aF0Eet48sx7Ft'

    userID = '@BarackObama'
    auth = tw.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tw.API(auth)
    tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count= 200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    
    last_tweet = tweets[0].full_text


    def prepro(last_tweet):
      ''' delete links, hashtags and tags from tweets'''
      regex1 = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
      regex2 = '\@[A-Za-z0-9.-]+'
      regex3 = '\B(\#[a-zA-Z]+\b)(?!;)'
      sentences = re.sub(regex1, '', last_tweet, flags=re.MULTILINE)
      sentences = re.sub(regex2, '', last_tweet, flags=re.MULTILINE)
      sentences = re.sub(regex3, '', last_tweet, flags=re.MULTILINE)

      phrases = tokenize.sent_tokenize(sentences) #transform tweet on a list of sentences


    def rephrasing(phrases):
      '''rephrase sentences'''
      with open('recap.txt', 'w') as f:
        for txt in phrases:
          print("-"*100)
          print("Input_phrase: ", txt)
          print("-"*100)
          f.write(txt)
          f.write('\n')
          para_phrases = parrot.augment(input_phrase=txt)
          for para_phrase in para_phrases:
            print(para_phrase)
            f.write(str(para_phrase))
            f.write('\n')

In [119]:
rephrase.scraping(userID)
rephrase.prepro(last_tweet)
rephrase.rephrasing(phrases)

AttributeError: ignored